In [1]:
# !git clone https://github.com/cg123/mergekit.git
# %cd mergekit
# %pip install -e .

In [2]:
# ! export TRANSFORMERS_CACHE="/dtu/blackhole/00/167776/cache"

In [3]:
import os

In [4]:
print("Current working directory:", os.getcwd())

Current working directory: /dtu/blackhole/00/167776/mergekit


In [5]:
OUTPUT_PATH = "./merged"  # folder to store the result in
LORA_MERGE_CACHE = "./merge_cache"  # change if you want to keep these for some reason
CONFIG_YML = "./examples/bio-merge.yml"  # merge configuration file
COPY_TOKENIZER = True  # you want a tokenizer? yeah, that's what i thought
LAZY_UNPICKLE = False  # experimental low-memory model loader
LOW_CPU_MEMORY = False  # enable if you somehow have more VRAM than RAM+swap

In [6]:
# actually do merge
import torch
import yaml

from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options=MergeOptions(
        lora_merge_cache=LORA_MERGE_CACHE,
        cuda=torch.cuda.is_available(),
        copy_tokenizer=COPY_TOKENIZER,
        lazy_unpickle=LAZY_UNPICKLE,
        low_cpu_memory=LOW_CPU_MEMORY,
    ),
)
print("Done!")

/dtu/blackhole/00/167776/DL/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
Warmup loader cache:   0%|                                                                                     | 0/3 [00:00<?, ?it/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Executing graph: 100%|███████████████████████████████████████████████████████████████████████████| 1748/1748 [32:20<00:00,  1.11s/it]


Done!


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset, load_dataset
import json
import sys


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("merged", cache_dir='./', device_map="cuda:0")
model = AutoModelForCausalLM.from_pretrained("merged", cache_dir='./', torch_dtype=torch.float16, device_map="cuda:0")

TypeError: __init__() got an unexpected keyword argument 'response_format'

In [28]:
# Define the prompt
prompt = """
Please answer the following question strictly in JSON format. Your response must contain only a JSON object 
with the keys "Explanation" and "Answer_choice". The "Answer_choice" should be one of three options: "A", "B", or "C". 
No extra text or explanation outside the JSON format is allowed.

Question: if x^2=9, then what is x?
A: 1
B: 2
C: 3
D: 5
"""
sent = tokenizer([prompt], return_tensors="pt").to("cuda:0")
output = model.generate(**sent, max_new_tokens=100, do_sample=True, pad_token_id=tokenizer.eos_token_id).to("cuda:0")
print(tokenizer.batch_decode(output)[0])

<s> 
Please answer the following question strictly in JSON format. Your response must contain only a JSON object 
with the keys "Explanation" and "Answer_choice". The "Answer_choice" should be one of three options: "A", "B", or "C". 
No extra text or explanation outside the JSON format is allowed.

Question: if x^2=9, then what is x?
A: 1
B: 2
C: 3
D: 5
E: 6

{ "Explanation": "Since x^2 = 9, we can find the value of x by taking the square root of both sides. 9 is a perfect square, so its square root is easily found to be 3. Therefore, the value of x is 3.", "Answer_choice": "C" }</s>


In [25]:
from huggingface_hub import InferenceClient
client = InferenceClient("merged", cache_dir='./', device_map="cuda:0")

TypeError: __init__() got an unexpected keyword argument 'cache_dir'